In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import psutil
import geopandas as gpd
import shapely
import shapefile
from shapely.geometry import Point
from collections import Counter
import folium
import requests
from bs4 import BeautifulSoup as BS

ModuleNotFoundError: No module named 'shapefile'

In [34]:
sf2009 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2009singlefamily_final_with_sqft.txt')
sf2013 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2013singlefamily_final_with_sqft.txt')
sf2017 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2017singlefamily_final_with_sqft.txt')
issued = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/Building_Permits_Issued.csv', low_memory=False)
parcels = pd.read_excel('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/zoneparcel.xlsx')
tax = pd.read_excel('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2016Tax_TN.xls')

In [6]:
rebuilt_parcels = parcels.loc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765877 entries, 0 to 765876
Data columns (total 7 columns):
ParcelID         765877 non-null int64
ZoneCode         765877 non-null object
Ordinance        761211 non-null object
IsActive         765877 non-null object
DateInactive     233307 non-null datetime64[ns]
APN              765877 non-null object
DateEffective    761203 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 40.9+ MB


In [ ]:
sf2009.Yearly_Housing_Spending.max()

In [23]:
sf2009.columns

Index(['APN', 'DistrictCode', 'DistrictNo', 'Address', 'City', 'Zip',
       'LandValue', 'ImprovementValue', 'AppraisedValue', '2009 TOTAL ASSD',
       'FinishedArea', 'Yearly_Housing_Spending', 'PricePerSqFt', 'bucket'],
      dtype='object')

In [24]:
zipgroup09 = sf2009.groupby('Zip')

In [27]:
zipgroup09.Yearly_Housing_Spending.mean()

Zip
37013     6429.827782
37015     5762.280660
37027    14238.488622
37064    18338.117403
37072     7788.142637
37076     7134.973407
37080     7909.650567
37086    10560.273773
37115     5644.746697
37122    10888.264902
37135    11998.900650
37138     8031.314720
37143     8454.964806
37189     7509.063384
37201     9883.906087
37203     8934.945052
37204    14772.760672
37205    23938.792544
37206     7114.811882
37207     4929.343087
37208     4696.789062
37209     7354.766848
37210     4717.537155
37211     6885.469945
37212    13274.998861
37214     6255.282583
37215    24071.307076
37216     6021.524041
37217     5465.004202
37218     5629.761241
37219     8731.559298
37220    19491.456963
37221     9637.674704
37228     2165.513026
Name: Yearly_Housing_Spending, dtype: float64

In [18]:
bucketgrouped09 = sf2009.groupby('bucket')

In [19]:
bucketgrouped09.Yearly_Housing_Spending.max()

bucket
(1) <30%_AMI         6115.988672
(2) 30-60%_AMI      12236.507706
(3) 60-80%_AMI      16318.363849
(4) 80-120%_AMI     24477.545773
(5) >120%_AMI      470115.662578
Name: Yearly_Housing_Spending, dtype: float64

In [20]:
districtgrouped09 = sf2009.groupby('DistrictNo')

In [21]:
districtgrouped09.Yearly_Housing_Spending.describe()

,count,mean,std,min,25%,50%,75%,max
DistrictNo,,,,,,,,
1,5902.0,6565.754903,4507.917105,185.744841,4178.126335,5635.770302,7981.365217,216895.610115
2,4340.0,4774.153910,1991.615614,906.072396,3479.318000,4435.224378,5726.377542,20454.584336
3,5311.0,6563.973316,4063.895011,1132.590495,4584.726323,5685.604284,7626.864392,131652.319115
4,6856.0,11991.756757,3908.915535,1041.983255,9101.497216,12204.795172,14492.627971,61354.692284
5,4508.0,4235.445213,1629.905755,819.995518,3225.617729,3877.989854,4865.608766,19100.006104
6,5201.0,8038.306362,3954.709731,906.072396,5513.450529,7062.834326,9767.460427,55265.885784
7,6143.0,6067.451924,2551.414759,906.072396,4534.892341,5735.438266,6926.923466,40818.561432
8,5278.0,5762.673655,2168.823524,942.315292,4267.600984,5341.296773,6849.907313,23449.153604
9,4808.0,5214.014143,2615.249053,1041.983255,3854.205454,4854.282861,5989.138536,41289.719078


In [22]:
districtgrouped09.Yearly_Housing_Spending.mean()

DistrictNo
1      6565.754903
2      4774.153910
3      6563.973316
4     11991.756757
5      4235.445213
6      8038.306362
7      6067.451924
8      5762.673655
9      5214.014143
10     6983.084925
11     7579.525395
12     7829.884059
13     5519.240556
14     5817.418489
15     6381.849091
16     5027.507436
17     8113.228755
18    13706.094612
19     8929.696809
20     5253.727602
21     5247.559666
22     8122.457601
23    25225.082624
24    15885.660275
25    17975.319020
26     7577.524664
27     6083.906004
28     5674.285735
29     5592.747944
30     5078.194547
31     8073.625556
32     6419.801242
33     6812.340203
34    28076.890178
35    10875.261531
Name: Yearly_Housing_Spending, dtype: float64

In [8]:
sf2009.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2009 LAND':'LandValue', '2009 IMPR':'ImprovementValue', '2009 TOTAL APPR':'AppraisedValue', '2009 TOTAL ASSD':'2009 TOTAL ASSD'}, inplace=True)
sf2013.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2013 LAND':'LandValue', '2013 IMPR':'ImprovementValue', '2013 TOTAL APPR':'AppraisedValue', '2013 TOTAL ASSD':'2013 TOTAL ASSD'}, inplace=True)
sf2017.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2017 LAND':'LandValue', '2017 IMPR':'ImprovementValue', '2017 TOTAL APPR':'AppraisedValue', '2017 TOTAL ASSD':'2017 TOTAL ASSD'}, inplace=True)

In [9]:
issued.rename(columns={'Permit #': 'PermitNo', 'Permit Type Description':'PermTypeDesc', 'Permit Subtype Description': 'PermSubDesc', 'Parcel':'Parcel', 'Date Entered':'EnteredDt', 'Date Issued': 'IssuedDt', 'Const. Cost': 'ConstCost', 'Address':'Address', 'City':'City', 'State':'State', 'Zip':'Zip', 'Subdivision/Lot': 'Subdiv_Lot', 'Contact':'Contact', 'Permit Type': 'PermType', 'Permit Subtype': 'PermSubtype', 'IVR Trk#': 'IVRTrk#', 'Purpose':'Purpose', 'Council Dist':'CouncilDist', 'Census Tract':'CensusTract', 'Mapped Location':'MappedLoc'}, inplace=True)

In [10]:
issued['EnteredDt'] = pd.to_datetime(issued['EnteredDt'])
issued['IssuedDt'] = pd.to_datetime(issued['IssuedDt'])

In [ ]:
issued.head()

In [11]:
nulls09 = sf2009.loc[sf2009['FinishedArea'] == 0].index
nulls13 = sf2013.loc[sf2013['FinishedArea'] == 0].index
nulls17 = sf2017.loc[sf2017['FinishedArea'] == 0].index

sf2009 = sf2009.drop(index=nulls09)
sf2013 = sf2013.drop(index=nulls13)
sf2017 = sf2017.drop(index=nulls17)

In [ ]:
sf2009

In [ ]:
issued.loc[issued.MappedLoc.str.match(r'\(') == True]

In [ ]:
issued.MappedLoc.searchsorted()

In [12]:
r=0.0475/12 # monthly interest rate
m=30*12 # number of months in 30 years

# calculate yearly spending

sf2009['Yearly_Housing_Spending']=12*(sf2009.AppraisedValue-sf2009.AppraisedValue*0.05)*((r*(1+r)**m)/((1+r)**(m-1)))
sf2013['Yearly_Housing_Spending']=12*(sf2013.AppraisedValue-sf2013.AppraisedValue*0.05)*((r*(1+r)**m)/((1+r)**(m-1)))
sf2017['Yearly_Housing_Spending']=12*(sf2017.AppraisedValue-sf2017.AppraisedValue*0.05)*((r*(1+r)**m)/((1+r)**(m-1)))

In [ ]:
issued.columns

In [13]:
sf2009['PricePerSqFt'] = sf2009.AppraisedValue/sf2009.FinishedArea
sf2013['PricePerSqFt'] = sf2013.AppraisedValue/sf2013.FinishedArea
sf2017['PricePerSqFt'] = sf2017.AppraisedValue/sf2017.FinishedArea

In [ ]:
sf2009.info()

In [ ]:
sf2009.head()

In [ ]:
sf2009.info()

In [ ]:
sf2009.loc[sf2009.Yearly_Housing_Spending == sf2009.Yearly_Housing_Spending.max()]

In [ ]:
print('30th percentile annual housing cost, 2009: ' + str(sf2009['Yearly_Housing_Spending'].quantile(0.3)))
print('30th percentile annual housing cost, 2013: ' + str(sf2013['Yearly_Housing_Spending'].quantile(0.3)))
print('30th percentile annual housing cost, 2017: ' + str(sf2017['Yearly_Housing_Spending'].quantile(0.3)))

# Creating buckets for affordability calculation

In [14]:
AMI=68000

income_cat_1=0.30*AMI
income_cat_2=0.60*AMI
income_cat_3=0.80*AMI
income_cat_4=1.20*AMI

In [15]:
cat1_spend=income_cat_1*0.3
cat2_spend=income_cat_2*0.3
cat3_spend=income_cat_3*0.3
cat4_spend=income_cat_4*0.3

print(cat1_spend, cat2_spend, cat3_spend, cat4_spend)

6120.0 12240.0 16320.0 24480.0


In [16]:
def bucket(x):
    if x<=cat1_spend:
        return '(1) <30%_AMI'
    elif x<=cat2_spend:
         return '(2) 30-60%_AMI'
    elif x<=cat3_spend:
        return '(3) 60-80%_AMI'
    elif x<=cat4_spend:
        return '(4) 80-120%_AMI'
    else:
        return '(5) >120%_AMI'

In [17]:

# assign the buckets to sf dataframes

sf2013['bucket']=sf2013.Yearly_Housing_Spending.apply(bucket)

sf2009['bucket']=sf2009.Yearly_Housing_Spending.apply(bucket)

sf2017['bucket']=sf2017.Yearly_Housing_Spending.apply(bucket)

In [33]:
sf2009.head()

,APN,DistrictCode,DistrictNo,Address,City,Zip,LandValue,ImprovementValue,AppraisedValue,2009 TOTAL ASSD,FinishedArea,Yearly_Housing_Spending,PricePerSqFt,bucket
0,00200000300,RT,10,1012 OLD BUCK HILL RD,GOODLETTSVILLE,37072,49800.0,65700.0,115500.0,28875.0,1920.0,5232.568086,60.156250,(1) <30%_AMI
1,00200000700,RT,10,1109 HIGHLAND AVE,GOODLETTSVILLE,37072,44100.0,88700.0,132800.0,33200.0,1464.0,6016.320708,90.710383,(1) <30%_AMI
2,00200001001002,GSD,10,0 SPRINGFIELD HWY,GOODLETTSVILLE,37072,15000.0,137900.0,152900.0,38225.0,3539.0,6926.923466,43.204295,(2) 30-60%_AMI
3,00200001400001,RT,10,133 B EAST END RD,GOODLETTSVILLE,37072,41100.0,27000.0,68100.0,17025.0,736.0,3085.176508,92.527174,(1) <30%_AMI
4,00200001600001,RT,10,141 B EAST END RD,GOODLETTSVILLE,37072,25100.0,60900.0,86000.0,21500.0,1301.0,3896.111302,66.102998,(1) <30%_AMI


In [ ]:
count13 = Counter(sf2013.bucket)
supply13 = dict(count13)

In [ ]:
count13

In [ ]:
count17 = Counter(sf2017.bucket)
supply17 = dict(count17)

In [ ]:
sf2009.loc[sf2009.Yearly_Housing_Spending == sf2009.Yearly_Housing_Spending.max()]

In [ ]:
sf2009.FinishedArea.describe()

In [ ]:
sf2009.loc[sf2009.PricePerSqFt == sf2009.PricePerSqFt.max()]

In [ ]:
sf2009.FinishedArea.plot(kind='hist')

In [ ]:
sf2009['Yearly_Housing_Spending'].describe()

In [ ]:
sf2013['Yearly_Housing_Spending'].describe()

In [ ]:
sf2017['Yearly_Housing_Spending'].describe()

In [ ]:
sf2009.Yearly_Housing_Spending.plot(kind='hist', bins=150)

In [ ]:
sf2013.info()

In [ ]:
sf2017.info()

In [ ]:
sf2009.AppraisedValue.describe()

In [ ]:
sf2009.AppraisedValue.plot()

In [ ]:
sf2013.AppraisedValue.describe()

In [ ]:
sf2013.AppraisedValue.plot()

In [ ]:
sf2017.AppraisedValue.describe()

In [ ]:
sf2017.AppraisedValue.plot()

In [ ]:
sf2009.AppraisedValue.describe()

In [ ]:
issued.head()

In [ ]:
issued.info()

In [ ]:
IssuedDt(demo) > IssuedDt(home) in issued where issued.PermType == CADM and issued.Address ==demos.Address

In [ ]:
dtdemissued = issued.loc[issued.PermType == 'CADM'].IssuedDt

In [ ]:
demos = issued.loc[issued.PermType == 'CADM']

In [ ]:
issued.loc[issued.Address.isin(demos.Address)]

In [ ]:
issued.loc[issued.isin(parcels)]

In [ ]:
issued.MappedLoc.unique()

In [ ]:
issued.columns

In [ ]:
issued.PermTypeDesc.unique()

In [ ]:
issued.head()

In [28]:
stypes = ['CAA01R301', 'CAA02R302', 'CAA03R298', 'CAA03R299', 'CAA03R398', 'CAA03R399']

In [ ]:
issued.loc[issued.Per]

In [29]:
homes = pd.DataFrame(issued.loc[issued.PermSubtype.isin(stypes)])

In [30]:
homes['CostToBuyer'] = homes.ConstCost * 0.09 + homes.ConstCost

In [31]:
r=0.0475/12 # monthly interest rate
m=30*12 # number of months in 30 years

homes['Yearly_Housing_Spending']=12*(homes.CostToBuyer-homes.CostToBuyer*0.05)*((r*(1+r)**m)/((1+r)**(m-1)))

In [32]:
homes['bucket']=homes.Yearly_Housing_Spending.apply(bucket)

In [50]:
income_by_zip_2016=pd.read_excel('./metrodata/2016Tax_TN.xls',skiprows=3)
income_by_zip_2016.tail()
income_by_zip_2016.rename(columns={'ZIP\ncode [1]':'Zip','Size of adjusted gross income':'IncomeBracket','Number of returns':'NumberofPeople','Adjusted gross income (AGI)':'AGI'},inplace=True)

In [49]:
income_by_zip_2016.columns

Index(['ZIP\ncode [1]', 'IncomeBracket', 'NumberofPeople',
       'Number of single returns', 'Number of joint returns',
       'Number of head of household returns',
       'Number with paid preparer's signature', 'Number of exemptions',
       'Number of dependents', 'Number of volunteer prepared returns [2]',
       ...
       'Total tax liability [10]', 'Unnamed: 136',
       'Total additional Medicare tax', 'Unnamed: 138',
       'Net investment income tax', 'Unnamed: 140',
       'Tax due at time of filing [11]', 'Unnamed: 142',
       'Overpayments refunded [12]', 'Unnamed: 144'],
      dtype='object', length=145)

In [51]:
income_by_zip_2016.head()

,Zip,IncomeBracket,NumberofPeople,Number of single returns,Number of joint returns,Number of head of household returns,Number with paid preparer's signature,Number of exemptions,Number of dependents,Number of volunteer prepared returns [2],...,Total tax liability [10],Unnamed: 136,Total additional Medicare tax,Unnamed: 138,Net investment income tax,Unnamed: 140,Tax due at time of filing [11],Unnamed: 142,Overpayments refunded [12],Unnamed: 144
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total,...,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount
1,NaN,NaN,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0,-8,...,-134,-135,-136,-137,-138,-139,-140,-141,-142,-143
2,0,Total,2950780.0,1314150.0,1110820.0,462700.0,1435540.0,5776740.0,1897310.0,56960,...,2204870,24604020,52900,127410,51540,214822,508740,2719722,2337700,6820816
3,0,"$1 under $25,000",1157130.0,744170.0,155040.0,238420.0,530230.0,1712970.0,580500.0,32710,...,558670,522263,0,0,0,0,100320,89425,1001430,2236479
4,0,"$25,000 under $50,000",759330.0,356940.0,217710.0,161440.0,347590.0,1486780.0,512360.0,18130,...,629120,1669448,0,0,0,0,104410,176633,646520,1784219


In [52]:
income_by_zip_2016.Zip.unique()

array([nan, 0, 37010, 37012, 37013, 37014, 37015, 37016, 37018, 37019,
       37020, 37022, 37023, 37025, 37026, 37027, 37028, 37029, 37030,
       37031, 37032, 37033, 37034, 37035, 37036, 37037, 37040, 37042,
       37043, 37046, 37047, 37048, 37049, 37050, 37051, 37052, 37055,
       37057, 37058, 37059, 37060, 37061, 37062, 37064, 37066, 37067,
       37069, 37072, 37073, 37074, 37075, 37076, 37078, 37079, 37080,
       37082, 37083, 37085, 37086, 37087, 37090, 37091, 37095, 37096,
       37097, 37098, 37101, 37110, 37115, 37118, 37122, 37127, 37128,
       37129, 37130, 37134, 37135, 37137, 37138, 37141, 37142, 37143,
       37144, 37145, 37146, 37148, 37149, 37150, 37151, 37153, 37160,
       37166, 37167, 37171, 37172, 37174, 37175, 37178, 37179, 37180,
       37181, 37183, 37184, 37185, 37186, 37187, 37188, 37189, 37190,
       37191, 37201, 37203, 37204, 37205, 37206, 37207, 37208, 37209,
       37210, 37211, 37212, 37214, 37215, 37216, 37217, 37218, 37219,
       37220, 37221

In [47]:

# remove blank rows and sub total per Zip Code
# AGI is in thousands of dollars

income_by_zip_2016=income_by_zip_2016[income_by_zip_2016['IncomeBracket']!='NaN']

In [48]:
income_by_zip_2016.

,ZIP code [1],IncomeBracket,NumberofPeople,Number of single returns,Number of joint returns,Number of head of household returns,Number with paid preparer's signature,Number of exemptions,Number of dependents,Number of volunteer prepared returns [2],...,Total tax liability [10],Unnamed: 136,Total additional Medicare tax,Unnamed: 138,Net investment income tax,Unnamed: 140,Tax due at time of filing [11],Unnamed: 142,Overpayments refunded [12],Unnamed: 144
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total,...,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount
1,NaN,NaN,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0,-8,...,-134,-135,-136,-137,-138,-139,-140,-141,-142,-143
2,0,Total,2950780.0,1314150.0,1110820.0,462700.0,1435540.0,5776740.0,1897310.0,56960,...,2204870,24604020,52900,127410,51540,214822,508740,2719722,2337700,6820816
3,0,"$1 under $25,000",1157130.0,744170.0,155040.0,238420.0,530230.0,1712970.0,580500.0,32710,...,558670,522263,0,0,0,0,100320,89425,1001430,2236479
4,0,"$25,000 under $50,000",759330.0,356940.0,217710.0,161440.0,347590.0,1486780.0,512360.0,18130,...,629120,1669448,0,0,0,0,104410,176633,646520,1784219


In [ ]:
bucketcount = Counter(homes.bucket)

In [ ]:
bucketcount

In [ ]:
count09

In [ ]:
count13

In [ ]:
count17

In [ ]:
homes.Yearly_Housing_Spending.describe()

In [ ]:
homes.head()

In [ ]:
homes.CostToBuyer.describe()

In [ ]:
homes.ConstCost.describe()

Certain permits have construction cost of 0.  These are anomalies (amended permits, master permits, etc.) that may be ignored.

In [ ]:
badpermits = homes.loc[homes.ConstCost == 0].index

In [ ]:
homes.drop(badpermits, inplace = True)

In [ ]:
rebuilds = pd.DataFrame(homes.loc[homes.Address.isin(demos.Address)])

In [ ]:
rebuilds.drop(rebuilds.loc[rebuilds['ConstCost'] == 0].index, inplace=True)

In [ ]:
rebuilds.ConstCost.describe()

In [ ]:
homes.ConstCost.describe()

In [ ]:
#rebuilds.set_index('IssuedDt', inplace=True)

In [ ]:
rebuilds.drop(rebuilds.loc[rebuilds.ConstCost > 3.678022e+05].index, inplace=True)

In [ ]:
rebuilds.info()

In [ ]:
homes.info()

In [ ]:
nonhomes = homes.loc[homes['ConstCost'] == 0.0].index

In [ ]:
homes.drop(index=nonhomes, inplace=True)

In [ ]:
permitdecode = pd.DataFrame(issued[['PermTypeDesc','PermType']])

In [ ]:
result = re.findall('-?\d*\.\d*', issued.MappedLoc[1])

In [ ]:
print(result)

In [ ]:
permitdecode.drop_duplicates()

CARN, CARR, CADM

In [ ]:
#resnew = pd.DataFrame(issued.loc[issued['PermType'] == 'CARW'])
respermits = pd.DataFrame(issued.loc[issued['PermType'].isin(['CARN','CARR'])])

In [ ]:
respermits.info()

In [ ]:
newhomes = respermits.Purpose.str.match(r'[dD]emolition')

In [ ]:
respermits.Purpose.unique()

In [ ]:
demos = pd.DataFrame(issued.loc[issued['PermType'] == 'CADM'])

In [ ]:
demos.info()

In [ ]:
issued.loc[issued['Address'] == '914 ACKLEN AVE']

In [ ]:
demos.head()

In [ ]:
demos.ConstCost.median()

In [ ]:
respermits.set_index('IssuedDt', inplace=True)

In [ ]:
respermits.info()

In [ ]:
len(respermits.PermSubDesc.unique())

In [ ]:
monthlycost = respermits['ConstCost'].resample('M').mean()

In [ ]:
monthlycost.plot()

In [ ]:
respermits.

In [ ]:
respermits.describe()

In [ ]:
resnew.EnteredDt.describe()

In [ ]:
resnew.IssuedDt.describe()

In [ ]:
resnew.ConstCost.describe()

In [ ]:
issued['CouncilDist'].unique()

In [ ]:
issued['MappedLoc'][1]

In [ ]:
issued['extcoords'] = issued.MappedLoc.str.extract(r'?\(.*\)', expand=False)

In [ ]:
coords = re.compile(r'\(.*\)')

In [ ]:
matches = []

In [ ]:
for string in issued.MappedLoc:
    match = re.findall(coords, string)
    matches.append(match)

In [ ]:
type(matches)

In [ ]:
type(matches[1])

In [ ]:
#[s for s in i[s] for i in matches]

coordlist = []
for i in enumerate(matches):
    i = []
    for s in i[s]:
        coordlist.append(tuple(s))

In [ ]:
len(matches)

In [ ]:
matches[1]

In [ ]:
issued['coords'] = pd.Series(matches).astype(tuple)

In [ ]:
issued['coords'][1]

Income data:
    Median average income: 68000
        Income subgroups:
        30% MAI (20400 /yr) (30% housing = 510/mo)
        60% MAI (40800 /yr) (30% housing = 1020/mo)
        80% MAI (54400 /yr) (30% housing = 1360/mo)
        120% MAI (81600 /yr) (30% housing = 2040/mo)

Model development:
    
    Owners = 54%
        - Model annual adjustments to these numbers
    
    
    Median Housing Cost - Owner = 1336/month * 12 = 16032/yr

    15-yr owner gap (affordable) = 3367
    15-yr owner gap (market) = 35751

    10-yr predicted owner gap (affordable) = 1257
    10-yr predicted owner gap (market) = 41421
    

In [ ]:
issued.Issued.min()

In [ ]:
issued.Issued.max()

In [ ]:
issued['monthlies'] = pd.Series([cost/360 for cost in issued.ConstCost])

In [ ]:
issued.info()

In [ ]:
issued.Zip.unique()

In [ ]:
r = requests.get('https://api.census.gov/data.html', auth=('davidwhainesiii@gmail.com', '83dffa3105cb08ef0e77c24ff28a1dc62272ae48'))

In [ ]:
type(r)

In [ ]:
rawresponse = r.text
censussoup = BS(rawresponse, 'html.parser')

In [ ]:
censussoup.find_all('table')

Question 1: At what point in time did the Codes Administration decide to begin “closing the loophole” by converting R zoning to RS?

Response 1: The Codes Administration does not set zoning policy, we interpret/enforce it. The Planning Commission and Metro Council are the ones who change zoning rules. The last major revision countywide to the zoning code was in 1997 when they rezoned a lot of properties in bulk. Since then, Councilmembers have proposed zoning changes for their neighborhoods on a piecemeal basis, or developers have requested zoning changes for particular properties. There hasn’t been an effort by Metro Government to downzone properties because of the duplex situation, but some Council members have. (edited)

Question 2: According to the report (Housing Nashville), there are predetermined amounts of needed housing and rental properties in each district. How is the split between housing demand and rental demand being evaluated? Is this arbitrary or is there census data giving us a proportion to go off of?

Question 3: What criteria was used to calculated the ‘Maximum Affordable Sales Price’  from the MHI on page 56 of the Housing Nashville Report? What interest rate and/or tax rate was assumed?

Response 2 & 3: Unfortunately, the folks that produced the Housing Nashville Report have left Metro Government so I don’t know that we’ll get all the answers. Looking at question three, I believe the estimate was based on a 5% down payment and about a 4% interest rate for a 30 year mortgage.
Also, We have also asked Assessments for an update to the 3-years of assessment data to include the square footage of the property and the zoning code for the property. (edited)

Renters = 46%
Median Housing Cost - Renter = 874/month * 12 = 10488/yr
    
    15-yr rental gap (affordable) = 17754
    15-yr rental gap (market) = 22219
        
    10-yr predicted rental gap (affordable) = 30934
    10-yr predicted rental gap (market) = 23043

In [ ]:
codetypes = pd.read_excel('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/Codes-Type-Subtype-List-20181011.xlsx')

In [ ]:
codetypes.head()

In [ ]:
codetypes.info()

In [ ]:
codetypes.CASE_TYPE_DESC.unique()

In [ ]:
codetypes.loc[codetypes.CASE_TYPE_DESC == 'Building Demolition Permit']

In [55]:
pwd

'/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi'

In [56]:
counties = gpd.read_file('./metrodata/cb_2017_us_zcta510_500k.shp')

CPLE_OpenFailedError: b'Unable to open ./metrodata/cb_2017_us_zcta510_500k.shx or ./metrodata/cb_2017_us_zcta510_500k.SHX.Try --config SHAPE_RESTORE_SHX true to restore or create it'